In [1]:
import json
import pickle
import plotly.express as px
import numpy as np
import os

In [2]:
# load location index
with open("data/akl_loc_idx.pkl", 'rb') as f:
    loc_idx = pickle.load(f) # datazone to point index
    idx_loc = {v:k for k, v in loc_idx.items()} # point index to datazone    
    print(f" -- loaded location index with dimension {len(loc_idx)}")

# load time index
with open("data/akl_t_idx.pkl", 'rb') as f:
    t_idx = pickle.load(f)
    print(f" -- loaded time index with dimension {len(t_idx)}")
    
# load precomputed odt
with open("data/akl_odt.npy", 'rb') as f:
    odt = np.load(f)
    print(f" -- loaded odt cube with dimensions {odt.shape}")

 -- loaded location index with dimension 2011
 -- loaded time index with dimension 72
 -- loaded odt cube with dimensions (2011, 2011, 72)


In [3]:
with open("data/akl_polygons.geojson", 'r') as f:
    polys = json.load(f)

In [4]:
# add id field to each feature (requried by plotly even though it's supposed to work via subproperty ref.)
for f in polys["features"]:
    f["id"] = f["properties"]["DZ2018"]        

In [ ]:
# remove properties DZ2018 as it is now duplicated
for f in polys["features"]:
    f["properties"] = f["properties"].pop("DZ2108", None)

In [ ]:
# combine polygons with duplicate feature ids
features = {}
for f in polys["features"]:
    fid = f["id"]
    if fid not in features:
        features[fid] = f        
    else:
        f_poly = f["geometry"]["coordinates"]
        features[fid]["geometry"]["coordinates"] += f_poly    
        #print(fid, len(features[fid]["geometry"]["coordinates"]))
polys["features"] = list(features.values())
print(f"total features: {len(polys['features'])}")

In [ ]:
# filter polygons by those that have population centroids
pids = [k for k, v in loc_idx.items()]
valid_features = [f for f in polys["features"] if f["id"] in pids]
polys["features"] = valid_features
print("total features:", len(polys["features"]))

In [ ]:
filter_valid = False

if filter_valid:
    # filter polygons by those that have valid journeys
    jsum = np.sum(np.nansum(odt, axis=2), axis=0)
    valid_idx = [i for i, v in enumerate(jsum) if v > 0]
    valid_features = [f for f in polys["features"] if loc_idx[f["id"]] in valid_idx]
    polys["features"] = valid_features
    print(f"total features: {len(polys['features'])}")

In [ ]:
# write indices
with open("../data/akl/akl_loc_idx.json", "w") as f:
    json.dump(loc_idx, f)
with open("../data/akl/akl_idx_loc.json", "w") as f:
    json.dump(idx_loc, f)    

In [ ]:
# save time index
idx_t = {v:int(k.timestamp())*1000 for k,v in t_idx.items()}
with open("../data/akl/akl_idx_t.json", "w") as f:
    json.dump(idx_t, f)

In [ ]:
#! scp -r ../data/akl/akl_idx_t.json transit:/home/ubuntu/transit_deprivation/data/akl

In [ ]:
# write file
with open("../data/akl/akl_polygons_id.geojson", "w") as f:
    json.dump(polys, f)

In [ ]:
# save a separate dt_slice file for each location
nan_value = -1
target_dir = "../data/akl"
if not os.path.exists(target_dir):
    os.mkdir(target_dir)

# replace nan values in odt
odt_nan = np.nan_to_num(odt, nan=nan_value)

fmt = np.vectorize(lambda x: f"{x:.1f}")
for f in polys["features"]:
    # origin id and odt index of origin
    o = f["id"]           
    o_idx = loc_idx[o]
    # get dt slice for the origin
    dt_slice = odt_nan[o_idx]            
    dt_slice = fmt(dt_slice).astype(np.float).tolist()
    # save the slice
    path = os.path.join(target_dir, f"{o}.json")
    with open(path, "w") as f:
        json.dump(dt_slice, f)        

In [ ]:
# data needs to be wgs84
locations = [f["id"] for f in polys["features"]]
fig = px.choropleth_mapbox(
        geojson=polys, 
        featureidkey="id",
        locations=locations,        
        center = {"lat": -36.8485, "lon": 174.7633},
        mapbox_style="carto-positron",
        zoom=12)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()